In [2]:
import requests
import re
import time
import json
from pprint import pprint
import random
import pandas as pd
import datetime

Функция для получения данных

In [3]:
proxy = {
    'http': 'http://46.47.197.210:3128',
    'http': 'http://62.33.207.201:80'
}

def get_api_data(url, params = '', headers ='', proxy = ''): 
    r = requests.get(url, params, headers=headers, proxies=proxy)
    return r.json()

Функция для случайного выбора юзер-агента

In [4]:
def choose_user_agent(user_agents):
    user_agent = user_agents[random.randrange(len(user_agents))]
    headers = {'user-agent': user_agent}
    return headers

Чтение данных юзерагетов

In [5]:
with open(r'C:\Users\Lenovo\Desktop\diplom\desktop_user_agent.txt', 'r', encoding='utf-8') as fp:
    user_agents = [line.strip() for line in fp.read().split('\n')]


Функция для получения данных о вакансиях за день

In [6]:
def get_vac_ids(date):
    url = r'https://api.hh.ru/vacancies'
    all_vac = []
    par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date)}
    data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
    pages = data['pages']
    for page in range(pages):
        par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date), 'page': page}
        data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
        all_vac = all_vac + data['items']
    vac_ids = []
    for one_vac in all_vac:
        vac_ids.append(one_vac['id'])
    vac_ids.sort()
    return vac_ids

Проверка нет ли капчи

In [32]:
url = f'https://api.hh.ru/vacancies/42018113'
vac_descr = get_api_data(url, headers=choose_user_agent(user_agents))
area = vac_descr['area']['name']
description = re.sub(r'\<[^>]*\>', '', vac_descr['description'])
employment = vac_descr['employment']['name']
experience = vac_descr['experience']['name']
id = vac_descr['id']
key_skills = vac_descr['key_skills']
if len(vac_descr['key_skills']) > 0:
    # if len(vac_descr['key_skills'][0]) > 0:
    skills = []
    for skill in vac_descr['key_skills']:
        skills.append(skill['name'])
    key_skills = skills
name = vac_descr['name']
publication_date = vac_descr['published_at'][0:10]
salary = vac_descr['salary']
schedule = vac_descr['schedule']
specializations = vac_descr['specializations']
if len(vac_descr['professional_roles']) > 0:
    prof_roles = []
    for role in vac_descr['professional_roles']:
        prof_roles.append(role['name'])
    proffesional_roles = prof_roles
employer = vac_descr['employer']['name']
print(salary)

None


Получение нужных данных о вакансиях

In [33]:
def get_vac_data(vac_ids):
    vac_info = []
    for one_vac in vac_ids:
        url = f'https://api.hh.ru/vacancies/{one_vac}'
        vac_descr = get_api_data(url, headers=choose_user_agent(user_agents))
        #print(vac_descr)
        try:
            area = vac_descr['area']['name']
            description = re.sub(r'\<[^>]*\>', '', vac_descr['description'])
            employment = vac_descr['employment']['name']
            experience = vac_descr['experience']['name']
            id = vac_descr['id']
            key_skills = vac_descr['key_skills']
            if len(vac_descr['key_skills']) > 0:
                skills = []
                for skill in vac_descr['key_skills']:
                    skills.append(skill['name'])
                key_skills = skills
            name = vac_descr['name']
            publication_date = vac_descr['published_at'][0:10]
            salary = vac_descr['salary']
            schedule = vac_descr['schedule']['name']
            specializations = vac_descr['specializations']
            if len(vac_descr['professional_roles']) > 0:
                prof_roles = []
                for role in vac_descr['professional_roles']:
                    prof_roles.append(role['name'])
                proffesional_roles = prof_roles
            employer = vac_descr['employer']['name']
        except:
            print(one_vac, 'ERRROR')
            continue
        vac_info.append({
            'area': area,
            'description': description,
            'employment': employment,
            'experience': experience,
            'id': id,
            'key_skills': key_skills,
            'name': name,
            'publication_date': publication_date,
            'salary': salary,
            'schedule': schedule,
            'specializations': specializations,
            'proffesional_roles': proffesional_roles,
            'employer': employer,
            'source': 'hh.ru'
        })
    return vac_info
    
    #time.sleep(0.1)


Функция для дописывания полученные данных за день в файл

In [9]:
def add_data_to_file(vac_info, path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)
    all_vacs = all_vacs + vac_info
    with open(path + '.json', 'w', encoding='utf-8') as fp:
        json.dump(all_vacs, fp, ensure_ascii=False)

Функция, чтобы сохранить из json в excel

In [10]:
def from_json_to_excel(path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        to_excel = json.load(fp)
    df = pd.DataFrame(to_excel)
    df.to_excel(path + '.xlsx')

Функция для добавления навыков в словарь и сохранения в файл (тут собираются уникальные значаения, этот в итоге не нужно)

In [11]:
def add_skills_to_dict(vac_info, save, path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        skills_dict = json.load(fp)
    for i, vac in enumerate(vac_info):
        if len(vac['key_skills']) > 0:
            for skill in vac['key_skills']:
                if skill.lower() not in skills_dict:
                    skills_dict.append(skill.lower())
    skills_dict.sort()
    if save == True:
        with open(path + '.json', 'w', encoding='utf-8') as fp:
            json.dump(skills_dict, fp, ensure_ascii=False)
    return skills_dict

In [106]:
ids = get_vac_ids(datetime.date.today() - datetime.timedelta(1))

In [107]:
vac_info = get_vac_data(ids)

In [108]:
print(vac_info[0])
print(len(vac_info))

{'area': 'Ульяновск', 'description': 'Привет, мы — MediaSoft.team! Мы разрабатываем сложные веб-системы, бэкенды, мобильные приложения и highload-проекты для бизнеса с 2014 года. У нас в команде 250+ разработчиков по направлениям backend, frontend, mobile, qa и аналитика. Наши офисы находятся в Ульяновске, Санкт-Петербурге, Самаре, Москве и Ростове-на-дону (FIRECODE). У нас точно хватит задач, которые покажутся тебе интересными. А в коллективе точно найдутся эксперты, которые знают больше. Мы предоставим тебе возможность развиваться в той области, которая тебе нравится. Мы уверены, что ты умеешь:  собирать и формулировать бизнес-требования; выявлять функциональные и нефункциональные требования из бизнес-требований; подготавливать техническую документацию (ГОСТы и стандарты, UML-диаграммы, семейство IDEF, BPMN и другие подходы).  Здорово, если ты знаешь SQL, СУБД, REST/SOAP, основы ООП и любого языка программирования. И еще лучше, если ты можешь и хочешь участвовать в выстраивании проце

In [109]:
path = r'C:\Users\Lenovo\Desktop\diplom\hh_data'
add_data_to_file(vac_info, path)

In [110]:
# path = r'C:\Users\Lenovo\Desktop\diplom\hh_data'
# from_json_to_excel(path)

In [111]:
#path = r'C:\Users\Lenovo\Desktop\diplom\skills_dict_orig'
#skills_dict = add_skills_to_dict(vac_info, True, path)